In [1]:
import re
from collections import defaultdict

In [2]:
db={
    "list some programming languages":["Python","Java","C"],
    "What is AI":["AI","stands","for","Artificial","Intelligence"]
}

In [37]:
word_embeddings={
    "what":0.2,
    "is":0.3,
    "ai":0.9,
    "list":0.7,
    "some":0.6,
    "programming":0.9,
    "languages":0.8
}

In [4]:
def normalize(text):
    text=text.lower()
    text=re.sub(r"[^\w\s]","",text)
    return text.strip()

In [5]:
def tokenize(text):
    return normalize(text).split()

In [6]:
def attention_score(w1,w2):
    return word_embeddings.get(w1,0.1)*word_embeddings.get(w2,0.1)

In [7]:
def encode_question(ques):
    tokens=tokenize(ques)
    focus_map={}
    for w1 in tokens:
        total=0.0
        for w2 in tokens:
            if w1!=w2:
                total+=attention_score(w1,w2)
        focus_map[w1]=total
    return tokens,focus_map
        

In [8]:
def find_best_match(tokens):
    token_set=set(tokens)
    best_key=None
    best_score=-1.0
    for x in db:
        x_set=set(tokenize(x))
        if not x and not token_set:
            continue
        intersection=token_set & x_set
        union=token_set | x_set
        score=len(intersection)/len(union) if union else 0.0
        if score>best_score:
            best_score=score
            best_key=x
    return best_key,best_score

In [24]:
def decoder(memory_key):
    if memory_key is None:
        return ["Answer","not","found!"]
    return db.get(memory_key,["Answer","not","found!"])

In [30]:
def transformer(q):
    tokens,focus_map=encode_question(q)
    print("--ENCODER--")
    print(f"Tokens:(tokens)")
    print("Focus Map:{focus_map}")
    print("--DECODER--")
    best_key,score=find_best_match(tokens)
    if score < 0.2:
        best_key = None
    print(f"Best Match: '{best_key}'(score: {score:.2f})")
    ans_tok=decoder(best_key)
    ans=" ".join(ans_tok)
    print(f"Generated Answer:{ans}")
    return ans

In [31]:
transformer("list some programming languages")

--ENCODER--
Tokens:(tokens)
Focus Map:{focus_map}
--DECODER--
Best Match: 'list some programming languages'(score: 1.00)
Generated Answer:Python Java C


'Python Java C'

In [32]:
transformer("What is AI")

--ENCODER--
Tokens:(tokens)
Focus Map:{focus_map}
--DECODER--
Best Match: 'What is AI'(score: 1.00)
Generated Answer:AI stands for Artificial Intelligence


'AI stands for Artificial Intelligence'

In [33]:
transformer("AI")

--ENCODER--
Tokens:(tokens)
Focus Map:{focus_map}
--DECODER--
Best Match: 'What is AI'(score: 0.33)
Generated Answer:AI stands for Artificial Intelligence


'AI stands for Artificial Intelligence'

In [34]:
transformer("What")

--ENCODER--
Tokens:(tokens)
Focus Map:{focus_map}
--DECODER--
Best Match: 'What is AI'(score: 0.33)
Generated Answer:AI stands for Artificial Intelligence


'AI stands for Artificial Intelligence'

In [35]:
transformer("Hi")

--ENCODER--
Tokens:(tokens)
Focus Map:{focus_map}
--DECODER--
Best Match: 'None'(score: 0.00)
Generated Answer:Answer not found!


'Answer not found!'

In [36]:
transformer("Hello there")

--ENCODER--
Tokens:(tokens)
Focus Map:{focus_map}
--DECODER--
Best Match: 'None'(score: 0.00)
Generated Answer:Answer not found!


'Answer not found!'